In [7]:
import pandas as pd

with open('/home/snt/projects_lujun/mt_reasoning/data/eng-ukr-Facebook-wikimatrix-1-eng-ukr/train-parts/Facebook-wikimatrix-1-eng-ukr.eng', 'r', encoding='utf-8') as f:
    eng_lines = [line.strip() for line in f]

with open('/home/snt/projects_lujun/mt_reasoning/data/eng-ukr-Facebook-wikimatrix-1-eng-ukr/train-parts/Facebook-wikimatrix-1-eng-ukr.ukr', 'r', encoding='utf-8') as f:
    ukr_lines = [line.strip() for line in f]

df = pd.DataFrame({
    'english': eng_lines,
    'ukrainian': ukr_lines
})

print(df.head())


                                             english  \
0                                   God is with us."   
1         O Reason, thou (alone) speakest the truth.   
2                        Are we not on Haq (truth)?"   
3              And said, 'Bring him in immediately.'   
4  She said, "The women are the ones who write th...   

                                           ukrainian  
0                                     Аллах з нами!»  
1  Наприклад, ти (tu) при ввічливому зверненні пе...  
2                    Ми не якась там попсова група!»  
3          Мухаммад сказав: - Покличте його до себе.  
4  Вона сказала: «Жінки — це ті, хто пише історію...  


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df["sentence_length"] = df["english"].str.len()

bins = [0, 10, 20, 30, 40, 50, 75, 100, 150, 200, 300, 500, 1000]
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]

df["length_range"] = pd.cut(df["sentence_length"], bins=bins, labels=labels, right=False)
length_counts = df["length_range"].value_counts().sort_index()

print(length_counts)


length_range
0-9         11002
10-19       75130
20-29      133154
30-39      187285
40-49      220324
50-74      554566
75-99      439359
100-149    539359
150-199    243560
200-299    137515
300-499     28820
500-999      4681
Name: count, dtype: int64


In [13]:
filtered_df = df[(df["sentence_length"] >= 200) & (df["sentence_length"] < 300)]
print(len(filtered_df))


137515


In [17]:
filtered_df

,english,translated_text,sentence_length,length_range,dataset,translation_source
2262,"Triana, Triana qué bonita está Triana, (Triana...","Triana, Triana qué bonita está Triana, (Тріана...",209,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
2617,In countries such as Greece and Spain many Bul...,"У таких країнах, як Греція та Іспанія, багато ...",218,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
4464,"For example, Fierlbeck (1998: 12) points out t...","Наприклад, Fierlbeck (1998: 12), вказує, що бі...",225,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
4792,2003 was the first time in fifty years that la...,"2003 року вперше за п'ятдесят років закони, що...",229,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
4841,According to the energy Progress Reports submi...,"Згідно зі звітами про енергетичну діяльність, ...",216,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
...,...,...,...,...,...,...
2576298,Both of these men were drawing from the rather...,"Вершники з'являються строго один за одним, кож...",210,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
2576334,"Historically, the offensive halfback was simil...","Коли спить, він зазвичай згортається, як кішка...",278,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
2576382,The film was voted one of the twelve greatest ...,У 1958 року фільм було включено до списку 12-т...,232,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook
2576387,This extravagance is consistent with their dem...,"Це заважає всім, хто зовні (з ворожими чи будь...",275,200-299,Facebook-wikimatrix-1-eng-ukr-filtered-from-20...,Facebook


In [18]:
from langchain.prompts import PromptTemplate


source_language_name = "English"
target_language_name = "Ukrainian"
source_language_abbreviation = "en"
target_language_abbreviation = "ukr"

filtered_df["dataset"] = "Facebook-wikimatrix-1-eng-ukr-filtered-from-200-to-299"
filtered_df["translation_source"] = "Facebook"

filtered_df.rename(columns={
    'english': 'source_text',
    'ukrainian': 'translated_text'
}, inplace=True)

def generate_translation_prompt(text, language_1, language_2):
    """Generate a translation prompt."""
    prompt_template = """Please translate the following {language_1} text into {language_2}. Please answer me with only translated text!

    ---------------------------------- Text to be translated ----------------------------------

    {text}

    ---------------------------------- Text to be translated ----------------------------------
    """
    translation_prompt = PromptTemplate(
        input_variables=["language_1", "language_2", "text"],
        template=prompt_template
    )
    return translation_prompt.format(language_1=language_1, language_2=language_2, text=text)

filtered_df["input"] = filtered_df["source_text"].apply(lambda x: generate_translation_prompt(x, source_language_name, target_language_name))
filtered_df["index"] = filtered_df.index

/tmp/ipykernel_506502/2795277482.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["dataset"] = "Facebook-wikimatrix-1-eng-ukr-filtered-from-200-to-299"
/tmp/ipykernel_506502/2795277482.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["translation_source"] = "Facebook"
/tmp/ipykernel_506502/2795277482.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [20]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    filtered_df,
    test_size=0.1,
    random_state=42, 
    shuffle=True
)

print(f"Train size: {len(train_df)}")
print(f"Val size: {len(val_df)}")


Train size: 123763
Val size: 13752


In [21]:

train_df.to_json(f"/home/snt/projects_lujun/mt_luxembourgish/data/datasets_general/dataset_train_{source_language_abbreviation}_{target_language_abbreviation}.jsonl", lines=True, orient="records")

In [22]:

val_df.to_json(f"/home/snt/projects_lujun/mt_luxembourgish/data/datasets_general/dataset_val_{source_language_abbreviation}_{target_language_abbreviation}.jsonl", lines=True, orient="records")